# Practice Data Analysis with Home Credit Competition dataset

In [4]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [5]:
!pip freeze > requirements.txt

In [6]:
# Dev simple functions
# Tính tỷ lệ giá trị khuyết thiếu
def calculate_missing_percentage(df=pd.DataFrame()):
    missing_value_percentage = []
    for col in df.columns:
        missing_value_percentage.append((col, df[col].isnull().sum() / len(df)))
    missing_value_percentage = sorted(missing_value_percentage, key=lambda x: x[1], reverse=True)
    missing_value_percentage = pd.DataFrame(missing_value_percentage, columns=['Feature', 'Missing Value Percentage'])
    return missing_value_percentage
# Vẽ box_plot
def plot_boxplot(df, column):
    plt.figure(figsize=(20, 6))
    sns.boxplot(x=df[column])
    plt.title(f'Box Plot of {column}')
    plt.xlabel(column)
    plt.show()
# Vẽ scatter plot
def plot_scatter(df, x_column, y_column, z_column=None):
    if z_column:
        plt.figure(figsize=(20, 6))
        sns.scatterplot(x=df[x_column], y=df[y_column], hue=df[z_column])
        plt.title(f'Scatter Plot of {x_column} vs {y_column} colored by {z_column}')
        plt.xlabel(x_column)
        plt.ylabel(y_column)
        plt.legend(title=z_column)
        plt.show()
    else:
        if x_column == y_column:
            raise ValueError("x_column and y_column cannot be the same for a scatter plot.")
# Vẽ count bar plot
def plot_bar(df, column):
    plt.figure(figsize=(20, 6))
    sns.countplot(x=df[column])
    plt.title(f'Bar Plot of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.show()
# Remove outliers using IQR
def remove_outliers_iqr(df, column, times=2.5):
    q1 = df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr_value = iqr(df[column])
    lower_bound = q1 - times * iqr_value
    upper_bound = q3 + times * iqr_value
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
# Draw multiple plots in one figure
# Biểu đồ histogram
def _plot_hist_subplot(x, fieldname, bins = 10, use_kde = True):
  x = x.dropna()
  xlabel = '{} bins tickers'.format(fieldname)
  ylabel = 'Count obs in {} each bin'.format(fieldname)
  title = 'histogram plot of {} with {} bins'.format(fieldname, bins)
  ax = sns.distplot(x, bins = bins, kde = use_kde)
  ax.set_xlabel(xlabel)
  ax.set_ylabel(ylabel)
  ax.set_title(title)
  return ax
# Biểu đồ barchart
def _plot_barchart_subplot(x, fieldname):
  xlabel = 'Group of {}'.format(fieldname)
  ylabel = 'Count obs in {} each bin'.format(fieldname)
  title = 'Barchart plot of {}'.format(fieldname)
  x = x.fillna('Missing')
  df_summary = x.value_counts(dropna = False)
  y_values = df_summary.values
  x_index = df_summary.index
  ax = sns.barplot(x = x_index, y = y_values, order = x_index)
  # Tạo vòng for lấy tọa độ đỉnh trên cùng của biểu đồ và thêm label thông qua annotate.
  labels = list(set(x))
  for label, p in zip(y_values, ax.patches):
    ax.annotate(label, (p.get_x()+0.25, p.get_height()+0.15))
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.title(title)
  return ax

# # Khởi tạo figure cho đồ thị (Kích thước W*H = 16x12) và cách nhau là 0.2 giữa các đồ thị
# fig = plt.figure(figsize=(18, 16))
# fig.subplots_adjust(hspace=0.5, wspace=0.2)
# # Tạo vòng for check định dạng của biến và visualize
# for i, (fieldname, dtype) in enumerate(zip(df.columns, df.dtypes.values)):
#   if i <= 11:
#     ax_i = fig.add_subplot(4, 3, i+1)
#     if dtype in ['float64', 'int64']:
#       ax_i = _plot_hist_subplot(df[fieldname], fieldname=fieldname)
#     else:
#       ax_i = _plot_barchart_subplot(df[fieldname], fieldname=fieldname)
      
# fig.suptitle('Visualization all fields')
# plt.show()

In [7]:
## Nhập dữ liệu
# path = Path.cwd().parent
# data_path = path /'data'/'.csv'
# df = pd.read_csv(data_path, low_memory=True)

In [3]:
## Import dữ liệu vào database

# import os
# import pandas as pd
# from sqlalchemy import create_engine

# server = ''
# database = ''

# conn_str = (
#     f"mssql+pyodbc://@{server}/{database}"
#     "?driver=ODBC+Driver+17+for+SQL+Server"
#     "&Trusted_Connection=yes"
#     "&Encrypt=no"
# )

# engine = create_engine(conn_str, fast_executemany=True)

# # Thư mục chứa file CSV
# folder = r"C:\"

# # Duyệt toàn bộ file CSV
# with engine.begin() as conn:
#     for file in os.listdir(folder):
#         if file.endswith(".csv"):
#             table_name = os.path.splitext(file)[0]
#             file_path = os.path.join(folder, file)
#             try:
#                 print(f"Importing '{file}' into table '{table_name}'...")
#                 df = pd.read_csv(file_path)
#                 df.to_sql(table_name, con=conn, index=False, if_exists='replace')
#                 print(f"Done '{table_name}'")
#             except Exception as e:
#                 print(f"Failed import '{file}': {e}")
#                 conn.rollback()  # QUAN TRỌNG: rollback nếu lỗi